In [2]:
from matplotlib import pyplot as plt
import pickle
import os
from history import History
import pandas as pd

In [3]:
HISTORY_DIR = "../history"

In [4]:
def create_df_from_history_ins(history_ins: int):
    FILE_PATH = f"{history_ins}_history.pkl"
    history_path = os.path.join(HISTORY_DIR, FILE_PATH)
    f = open(history_path, "rb")
    history = pickle.load(f)
    player_1_reward = history.get("player_1_reward")
    final_state = history.get("final_state")
    done_type = history.get("done_type")
    winner = history.get("winner")

    print(len(player_1_reward), len(final_state), len(done_type), len(winner))

    df = pd.DataFrame({
        "player_1_reward": player_1_reward,
        "final_state": final_state,
        "done_type": done_type,
        "winner": winner
    })
    return df


# Training agent against random agent for 300 episodes

In [5]:
df = create_df_from_history_ins(6)
df2 = df.drop(columns=['final_state'])

450 450 450 450


2022-05-31 09:34:06.538669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Most of all the wins are **vertical wins**

Still a lot of **illegal_moves** -> Hasn't learned to not play correctly, migth be player_2

In [126]:
df2['done_type'].value_counts()

vertical         298
illegal_moves     89
horizontal        51
diagonal          12
Name: done_type, dtype: int64

Agent 1, who also starts, wins 54.3% of the games not being ended due to illegal_moves

In [127]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    196
2    165
0     89
Name: winner, dtype: int64


1    0.542936
2    0.457064
0    0.246537
Name: winner, dtype: float64

40 of the 89 *illegal_moves*, 45%, are done by player 1.

In [130]:
illegal = df2[(df2['done_type'] == 'illegal_moves') & (df2['player_1_reward'] == -1)].count()
illegal_percent = illegal / len(df2[df2['done_type'] == 'illegal_moves'])
print(illegal)
illegal_percent


player_1_reward    40
done_type          40
winner             40
dtype: int64


player_1_reward    0.449438
done_type          0.449438
winner             0.449438
dtype: float64

# Run 1000 episodes w/training_agent starting with weights main_6_200 and opponent main_6_100

In [131]:
history_instance = 7
df = create_df_from_history_ins(history_instance)
df2 = df.drop(columns=['final_state'])

1100 1100 1100 1100


In [132]:
df2['done_type'].value_counts()

vertical         735
horizontal       168
illegal_moves    155
diagonal          42
Name: done_type, dtype: int64

In [133]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    554
2    391
0    155
Name: winner, dtype: int64


1    0.586243
2    0.413757
0    0.164021
Name: winner, dtype: float64

Player 1 only made 40% of the illegal_moves

In [135]:
illegal = df2[(df2['done_type'] == 'illegal_moves') & (df2['player_1_reward'] == -1)].count()
illegal_percent = illegal / len(df2[df2['done_type'] == 'illegal_moves'])
print(illegal)
illegal_percent


player_1_reward    63
done_type          63
winner             63
dtype: int64


player_1_reward    0.406452
done_type          0.406452
winner             0.406452
dtype: float64

In [139]:
df2[['player_1_reward']].value_counts()

player_1_reward
 1                 554
-1                 454
 0                  92
dtype: int64

# 3000 episodes, opponent starts with policy 900 episodes prior

In [140]:
history_instance = 8
df = create_df_from_history_ins(history_instance)
df2 = df.drop(columns=['final_state'])

3110 3110 3110 3110


In [141]:
df2['done_type'].value_counts()

vertical         2375
illegal_moves     360
horizontal        335
diagonal           40
Name: done_type, dtype: int64

In [142]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    2194
2     556
0     360
Name: winner, dtype: int64


1    0.797818
2    0.202182
0    0.130909
Name: winner, dtype: float64

In [143]:
illegal = df2[(df2['done_type'] == 'illegal_moves') & (df2['player_1_reward'] == -1)].count()
illegal_percent = illegal / len(df2[df2['done_type'] == 'illegal_moves'])
print(illegal)
illegal_percent


player_1_reward    175
done_type          175
winner             175
dtype: int64


player_1_reward    0.486111
done_type          0.486111
winner             0.486111
dtype: float64

# Test 500 episodes against random agent

In [147]:
history_instance = 9
df = create_df_from_history_ins(history_instance)
df2 = df.drop(columns=['final_state'])

500 500 500 500


In [148]:
df2['done_type'].value_counts()

vertical         432
illegal_moves     43
horizontal        23
diagonal           2
Name: done_type, dtype: int64

The trained agent wins 82.5% of the times against the random agent

In [149]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    377
2     80
0     43
Name: winner, dtype: int64


1    0.824945
2    0.175055
0    0.094092
Name: winner, dtype: float64

# Training agent for 1000 episodes

In [155]:
df = create_df_from_history_ins(10)
df2 = df.drop(columns=['final_state'])

1100 1100 1100 1100


In [156]:
df2['done_type'].value_counts()

vertical         891
illegal_moves     92
horizontal        70
diagonal          47
Name: done_type, dtype: int64

In [157]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    820
2    188
0     92
Name: winner, dtype: int64


1    0.813492
2    0.186508
0    0.091270
Name: winner, dtype: float64

In [159]:
illegal = df2[(df2['done_type'] == 'illegal_moves') & (df2['player_1_reward'] == -1)].count()
illegal_percent = illegal / len(df2[df2['done_type'] == 'illegal_moves'])
print(illegal)
illegal_percent

player_1_reward    42
done_type          42
winner             42
dtype: int64


player_1_reward    0.456522
done_type          0.456522
winner             0.456522
dtype: float64

The agent seems to overfit to only trying vertical wins. Change the rewards

# Changed agent rewards to restrict vertical wins and illegal moves

In [160]:
df = create_df_from_history_ins(11)
df2 = df.drop(columns=['final_state'])

2100 2100 2100 2100


In [161]:
df2['done_type'].value_counts()

vertical         1975
illegal_moves      51
horizontal         47
diagonal           27
Name: done_type, dtype: int64

Agent wins 96% of all the games against a prior policy of itself. Almost all are vertical, despite try to increase horizontal/diagonal wins

In [162]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    1972
2      77
0      51
Name: winner, dtype: int64


1    0.962421
2    0.037579
0    0.024890
Name: winner, dtype: float64

### Never choose illegal_moves > 1 time!
It seems as if the agent has learned not to choose illegal moves > 1 time

In [181]:
illegal_move_reward = -5
df2[(df2['done_type'] == 'illegal_moves') & (df2['player_1_reward'] <= illegal_move_reward)]

,player_1_reward,done_type,winner


# Agent trained on a opponent lagging 2 * 300 episodes behind for 3000 episodes

In [7]:
df = create_df_from_history_ins(12)
df2 = df.drop(columns=['final_state'])

3200 3200 3200 3200


In [8]:
df2['done_type'].value_counts()

vertical         1919
illegal_moves     740
horizontal        377
diagonal          164
Name: done_type, dtype: int64

In [9]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    1572
2     888
0     740
Name: winner, dtype: int64


1    0.639024
2    0.360976
0    0.300813
Name: winner, dtype: float64

In [10]:
illegal_move_reward = -5
df2[df2['player_1_reward'] <= -1]

,player_1_reward,done_type,winner
1,-1.0,diagonal,2
2,-1.0,horizontal,2
3,-1.0,vertical,2
8,-1.0,vertical,2
11,-1.0,diagonal,2
...,...,...,...
3018,-1.0,vertical,2
3021,-1.0,vertical,2
3045,-1.0,vertical,2
3048,-1.0,vertical,2


In [11]:
# Get the average reward
df2['player_1_reward'].mean()

0.214765625

# ~15000 episodes with opponent lagging 3 policies behind
Because I had to abort the training and had falsy added history-checkpoint, the history's corrupted. Instead I tested the agent against a random player for 600 episodes

In [12]:
df = create_df_from_history_ins(14)
df2 = df.drop(columns=['final_state'])

610 610 610 610


In [13]:
df2['done_type'].value_counts()

vertical         404
illegal_moves    148
horizontal        36
diagonal          22
Name: done_type, dtype: int64

In [14]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_moves']))
win_percent

1    402
0    148
2     60
Name: winner, dtype: int64


1    0.870130
0    0.320346
2    0.129870
Name: winner, dtype: float64

In [15]:
illegal_move_reward = -5
df2[df2['player_1_reward'] <= illegal_move_reward]

,player_1_reward,done_type,winner


# Trained agent for ~4000 episodes after fixing illegal_action-reward
Vertical victory is also reduced to a reward of 0.5, instead of 0.75

In [16]:
df = create_df_from_history_ins(15)
df2 = df.drop(columns=['final_state'])

3760 3760 3760 3760


In [17]:
df2['done_type'].value_counts()

vertical        2266
horizontal       774
illegal_move     595
diagonal         125
Name: done_type, dtype: int64

Kept training the agent for ~6200 more episodes

In [18]:
df = create_df_from_history_ins(16)
df2 = df.drop(columns=['final_state'])

6200 6200 6200 6200


There is an increase in horizontal wins, but vertical's still superior

In [19]:
df2['done_type'].value_counts()

vertical        4317
illegal_move    1103
horizontal       722
diagonal          58
Name: done_type, dtype: int64

In [25]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_move']))
win_percent

2    2781
1    2316
0    1103
Name: winner, dtype: int64


2    0.545615
1    0.454385
0    0.216402
Name: winner, dtype: float64

336 illegal moves, not good

In [238]:
illegal_move_reward = -5
df2[df2['player_1_reward'] <= illegal_move_reward].count()

player_1_reward    336
done_type          336
winner             336
dtype: int64

Although the results doesn't seem that good, it's a lot better when you play against it than before! Let's try it against a random agent

In [239]:
df = create_df_from_history_ins(17)
df2 = df.drop(columns=['final_state'])

610 610 610 610


In [240]:
df2['done_type'].value_counts()

vertical        352
horizontal      142
illegal_move     79
diagonal         37
Name: done_type, dtype: int64

In [26]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_move']))
win_percent

2    2781
1    2316
0    1103
Name: winner, dtype: int64


2    0.545615
1    0.454385
0    0.216402
Name: winner, dtype: float64

The trained agent only wins 56% of the games against the random agent. Does it do better against an earlier version of itself? This might have been with weight_13 and not weight_14 as wanted

# Trained overnight with same setup as over

In [27]:
df = create_df_from_history_ins(18)
df2 = df.drop(columns=['final_state'])

24500 24500 24500 24500


In [28]:
df2['done_type'].value_counts()

vertical        20701
illegal_move     3113
horizontal        634
diagonal           52
Name: done_type, dtype: int64

In [29]:
print(df2['winner'].value_counts())
win_percent = df2['winner'].value_counts() / (len(df2) - len(df2[df['done_type'] == 'illegal_move']))
win_percent

1    13379
2     8008
0     3113
Name: winner, dtype: int64


1    0.625567
2    0.374433
0    0.145556
Name: winner, dtype: float64

The agent unfortunately overfits to the vertical strategy, it doesn't seem to have learned that it shouldn't take illegal moves neither.
Training a new agent from scratch. Increasing the depth with 1 dense layer. Removing the possibility to choose a wrong action once -> Directly ends if wrong is chosen